This notebook aims to assess the generalization accuracy of a generated suffix, assuming a data-split was used during training.

In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import torch
from torch import nn
import matplotlib.pyplot as plt
from copy import deepcopy
import pandas as pd
from tqdm import tqdm
from collections import defaultdict
from transformers import AutoTokenizer
import pandas as pd
import seaborn as sns
from types import SimpleNamespace
from datasets import Dataset
from os.path import join as oj
import pickle as pkl
import os
import dvu
dvu.set_style()
import analyze_utils
import sys
sys.path.append('..')
import data
from model_utils import prompt_classification

# Load the results + get generated suffixes

In [6]:
# NOTE: SAVED THIS TO A DIFFERENT DIR!!!!
results_acc_dir = oj('/home/chansingh/interpretable-autoprompting/results/generalization_acc/accs_sent')
accs = [
    pd.DataFrame.from_dict(pkl.load(open(oj(results_acc_dir, d), 'rb')))
    for d in os.listdir(results_acc_dir)
]
df = pd.concat(accs)
df['max_digit'] = df['max_digit'].fillna(10)
df = df.drop_duplicates()

# extract acc since gpt3 func returns a tuple
def extract_acc(x):
    if isinstance(x, tuple):
        return x[1]
    else:
        return x
df.acc = df.acc.apply(extract_acc)

FileNotFoundError: [Errno 2] No such file or directory: '/home/chansingh/interpretable-autoprompting/results/generalization_acc/accs_sent'

In [5]:
task_names_sentiment = ['ffb_train', 'imdb_train', 'rt_train', 'sst2_train', 'tweets_train']
d = df[(df.task_name.isin(task_names_sentiment))]
d.head()

,checkpoint,prompt,task_name,n_shots,max_digit,train_split_frac,prompt_actual,acc,task_name_prompt,prompt_seed
0,EleutherAI/gpt-j-6B,iprompt,ffb_train,1,10,None,life Answer: Yes (because it's about life) ...,0.333333,sst2_train,1.0
0,EleutherAI/gpt-j-6B,manual,tweets_train,1,10,None,Answer Yes if the input is hate speech and No ...,5.333333,tweets_train,1.0
0,EleutherAI/gpt-j-6B,iprompt,ffb_train,1,10,None,Answer: No Answer: No Answer: No Answer: No A...,2.333333,tweets_train,1.0
0,gpt3,,imdb_train,1,10,None,,1.000000,imdb_train,1.0
0,EleutherAI/gpt-j-6B,iprompt,tweets_train,1,10,None,This was filmed back-to-back with the 1992 re...,1.000000,imdb_train,1.0
